In [19]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
import intake 
import pprint
from src.LE_LoadAndMerge import *
import warnings
warnings.filterwarnings("ignore")
import sys  
sys.path.insert(0, '/home/jupyter/InternalVariability/AdaptationAnalysis')
from app.main.src.utils import *
import datetime


In [2]:
def drop_bounds_height(ds):
        
    """Drop coordinates like 'time_bounds' from datasets,
    which can lead to issues when merging."""
    drop_vars = [vname for vname in ds.coords
            if (('_bounds') in vname ) or ('_bnds') in vname or ('height') in vname]
    return ds.drop(drop_vars)

In [3]:
url = 'https://storage.googleapis.com/cmip6/pangeo-cmip6.json'
raw_cat = intake.open_esm_datastore(url)

### ACCESS

In [4]:
cat = raw_cat.search(
    experiment_id=['historical','ssp585'],
    variable_id='tas',
    table_id = 'day',
    source_id ='ACCESS-ESM1-5'
)

In [5]:
# load data into xarray datasets
dset = cat.to_dataset_dict(zarr_kwargs={'consolidated':True}, storage_options={"anon": True});


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


In [6]:
keys = sorted(dset.keys())

In [7]:
hist = dset[keys[0]]
future = dset[keys[1]]

In [8]:
lat=41.3
lon=-72.5
lon = lon % 360

In [9]:
hist = hist.sel(lat=lat,lon=lon,method='nearest')
future = future.sel(lat=lat,lon=lon,method='nearest')

In [10]:
hist = hist.chunk({'member_id': 1, 'time': -1})
future = future.chunk({'member_id': 1, 'time': -1})

In [11]:
%%time
ds_future = []
for member_id in future.member_id:
    ds = future.sel(member_id = member_id).load()
    ds_future.append(ds)
ACCESS_future = xr.concat(ds_future,'member_id')

CPU times: user 4min 9s, sys: 1min 19s, total: 5min 28s
Wall time: 2min 56s


In [12]:
%%time
ds_hist = []
for member_id in hist.member_id:
    ds = hist.sel(member_id = member_id).load()
    ds_hist.append(ds)
ACCESS_hist = xr.concat(ds_hist,'member_id')

CPU times: user 7min 47s, sys: 2min 18s, total: 10min 6s
Wall time: 5min 25s


In [13]:
if ACCESS_hist.lon > 180 or ACCESS_hist.lon < -180:
    # convert lon from 0-360 to -180 to 180
    ACCESS_hist = ACCESS_hist.assign_coords(lon=((ACCESS_hist.lon + 180) % 360 - 180))
else: 
    pass 

if ACCESS_future.lon > 180 or ACCESS_future.lon < -180:
    # convert lon from 0-360 to -180 to 180
    ACCESS_future = ACCESS_future.assign_coords(lon=((ACCESS_future.lon + 180) % 360 - 180))
else: 
    pass 

In [14]:
ACCESS_hist = drop_bounds_height(ACCESS_hist)
ACCESS_future = drop_bounds_height(ACCESS_future)

In [15]:
# hist has 30 members and all have data 
ACCESS_hist

<xarray.Dataset>
Dimensions:    (member_id: 30, time: 60265)
Coordinates:
    lat        float64 41.25
    lon        float64 -73.12
  * time       (time) datetime64[ns] 1850-01-01T12:00:00 ... 2014-12-31T12:00:00
  * member_id  (member_id) <U9 'r1i1p1f1' 'r2i1p1f1' ... 'r24i1p1f1' 'r22i1p1f1'
Data variables:
    tas        (member_id, time) float32 270.3 274.2 282.8 ... 286.7 276.9 273.2
Attributes: (12/44)
    source:                  ACCESS-ESM1.5 (2019): \naerosol: CLASSIC (v1.0)\...
    branch_method:           standard
    Conventions:             CF-1.7 CMIP-6.2
    forcing_index:           1
    version:                 v20200922
    run_variant:             forcing: GHG, Oz, SA, Sl, Vl, BC, OC, (GHG = CO2...
    ...                      ...
    variable_id:             tas
    initialization_index:    1
    institution_id:          CSIRO
    activity_id:             CMIP
    branch_time_in_child:    0.0
    intake_esm_dataset_key:  CMIP.CSIRO.ACCESS-ESM1-5.historical.day.gn

In [17]:
np.isfinite(ACCESS_hist.tas).sum('time')

<xarray.DataArray 'tas' (member_id: 30)>
array([60265, 60265, 60265, 60265, 60265, 60265, 60265, 60265, 60265,
       60265, 60265, 60265, 60265, 60265, 60265, 60265, 60265, 60265,
       60265, 60265, 60265, 60265, 60265, 60265, 60265, 60265, 60265,
       60265, 60265, 60265])
Coordinates:
    lat        float64 41.25
    lon        float64 -73.12
  * member_id  (member_id) <U9 'r1i1p1f1' 'r2i1p1f1' ... 'r24i1p1f1' 'r22i1p1f1'

In [16]:
# future has 10 members, but some of them start with nans 
# this may have to do with time going out to 2300 - maybe some members dont kick in until later?? 
ACCESS_future

<xarray.Dataset>
Dimensions:    (time: 104459, member_id: 10)
Coordinates:
  * time       (time) object 2015-01-01 12:00:00 ... 2300-12-31 12:00:00
    lat        float64 41.25
    lon        float64 -73.12
  * member_id  (member_id) <U9 'r2i1p1f1' 'r1i1p1f1' ... 'r10i1p1f1' 'r8i1p1f1'
Data variables:
    tas        (member_id, time) float32 nan nan nan nan ... 291.3 292.6 295.8
Attributes: (12/43)
    branch_time_in_parent:   60265.0
    source:                  ACCESS-ESM1.5 (2019): \naerosol: CLASSIC (v1.0)\...
    branch_method:           standard
    Conventions:             CF-1.7 CMIP-6.2
    forcing_index:           1
    run_variant:             forcing: GHG, Oz, SA, Sl, Vl, BC, OC, (GHG = CO2...
    ...                      ...
    variable_id:             tas
    initialization_index:    1
    institution_id:          CSIRO
    activity_id:             ScenarioMIP
    branch_time_in_child:    60265.0
    intake_esm_dataset_key:  ScenarioMIP.CSIRO.ACCESS-ESM1-5.ssp585.day.gn

In [18]:
np.isfinite(ACCESS_future.tas).sum('time')

<xarray.DataArray 'tas' (member_id: 10)>
array([ 36524, 104459,  18262, 104459, 104459,  36524,  36524, 104459,
       104459,  18262])
Coordinates:
    lat        float64 41.25
    lon        float64 -73.12
  * member_id  (member_id) <U9 'r2i1p1f1' 'r1i1p1f1' ... 'r10i1p1f1' 'r8i1p1f1'

once we slice future time, we get all nans for some of the members

In [19]:
ACCESS_future_sliced = ACCESS_future.sel(time=slice('2015','2100'))
ACCESS_hist_sliced = ACCESS_hist.sel(time=slice('1920','2014'))

In [20]:
ACCESS_future_sliced

<xarray.Dataset>
Dimensions:    (time: 31411, member_id: 10)
Coordinates:
  * time       (time) object 2015-01-01 12:00:00 ... 2100-12-31 12:00:00
    lat        float64 41.25
    lon        float64 -73.12
  * member_id  (member_id) <U9 'r2i1p1f1' 'r1i1p1f1' ... 'r10i1p1f1' 'r8i1p1f1'
Data variables:
    tas        (member_id, time) float32 nan nan nan nan nan ... nan nan nan nan
Attributes: (12/43)
    branch_time_in_parent:   60265.0
    source:                  ACCESS-ESM1.5 (2019): \naerosol: CLASSIC (v1.0)\...
    branch_method:           standard
    Conventions:             CF-1.7 CMIP-6.2
    forcing_index:           1
    run_variant:             forcing: GHG, Oz, SA, Sl, Vl, BC, OC, (GHG = CO2...
    ...                      ...
    variable_id:             tas
    initialization_index:    1
    institution_id:          CSIRO
    activity_id:             ScenarioMIP
    branch_time_in_child:    60265.0
    intake_esm_dataset_key:  ScenarioMIP.CSIRO.ACCESS-ESM1-5.ssp585.day.gn

In [22]:
np.isfinite(ACCESS_future_sliced.tas).sum('time')

<xarray.DataArray 'tas' (member_id: 10)>
array([    0, 31411,     0, 31411, 31411,     0,     0, 31411, 31411,
           0])
Coordinates:
    lat        float64 41.25
    lon        float64 -73.12
  * member_id  (member_id) <U9 'r2i1p1f1' 'r1i1p1f1' ... 'r10i1p1f1' 'r8i1p1f1'

In [21]:
ACCESS_hist_sliced

<xarray.Dataset>
Dimensions:    (member_id: 30, time: 34699)
Coordinates:
    lat        float64 41.25
    lon        float64 -73.12
  * time       (time) datetime64[ns] 1920-01-01T12:00:00 ... 2014-12-31T12:00:00
  * member_id  (member_id) <U9 'r1i1p1f1' 'r2i1p1f1' ... 'r24i1p1f1' 'r22i1p1f1'
Data variables:
    tas        (member_id, time) float32 275.4 278.1 279.1 ... 286.7 276.9 273.2
Attributes: (12/44)
    source:                  ACCESS-ESM1.5 (2019): \naerosol: CLASSIC (v1.0)\...
    branch_method:           standard
    Conventions:             CF-1.7 CMIP-6.2
    forcing_index:           1
    version:                 v20200922
    run_variant:             forcing: GHG, Oz, SA, Sl, Vl, BC, OC, (GHG = CO2...
    ...                      ...
    variable_id:             tas
    initialization_index:    1
    institution_id:          CSIRO
    activity_id:             CMIP
    branch_time_in_child:    0.0
    intake_esm_dataset_key:  CMIP.CSIRO.ACCESS-ESM1-5.historical.day.gn

In [23]:
np.isfinite(ACCESS_hist_sliced.tas).sum('time')

<xarray.DataArray 'tas' (member_id: 30)>
array([34699, 34699, 34699, 34699, 34699, 34699, 34699, 34699, 34699,
       34699, 34699, 34699, 34699, 34699, 34699, 34699, 34699, 34699,
       34699, 34699, 34699, 34699, 34699, 34699, 34699, 34699, 34699,
       34699, 34699, 34699])
Coordinates:
    lat        float64 41.25
    lon        float64 -73.12
  * member_id  (member_id) <U9 'r1i1p1f1' 'r2i1p1f1' ... 'r24i1p1f1' 'r22i1p1f1'

In [24]:
# only left with 5 ensemble members after slicing data and dropping members with nans
ACCESS_future_sliced.dropna(dim='member_id')

<xarray.Dataset>
Dimensions:    (time: 31411, member_id: 5)
Coordinates:
  * time       (time) object 2015-01-01 12:00:00 ... 2100-12-31 12:00:00
    lat        float64 41.25
    lon        float64 -73.12
  * member_id  (member_id) <U9 'r1i1p1f1' 'r4i1p1f1' ... 'r9i1p1f1' 'r10i1p1f1'
Data variables:
    tas        (member_id, time) float32 282.4 289.8 278.7 ... 280.7 279.4 277.9
Attributes: (12/43)
    branch_time_in_parent:   60265.0
    source:                  ACCESS-ESM1.5 (2019): \naerosol: CLASSIC (v1.0)\...
    branch_method:           standard
    Conventions:             CF-1.7 CMIP-6.2
    forcing_index:           1
    run_variant:             forcing: GHG, Oz, SA, Sl, Vl, BC, OC, (GHG = CO2...
    ...                      ...
    variable_id:             tas
    initialization_index:    1
    institution_id:          CSIRO
    activity_id:             ScenarioMIP
    branch_time_in_child:    60265.0
    intake_esm_dataset_key:  ScenarioMIP.CSIRO.ACCESS-ESM1-5.ssp585.day.gn

### EC-Earth3

In [20]:
cat = raw_cat.search(
    experiment_id=['historical','ssp585'],
    variable_id='tas',
    table_id = 'day',
    source_id ='EC-Earth3'
)

In [21]:
cat.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,ScenarioMIP,EC-Earth-Consortium,EC-Earth3,ssp585,r6i1p1f1,day,tas,gr,gs://cmip6/CMIP6/ScenarioMIP/EC-Earth-Consorti...,NaN,20200201
1,ScenarioMIP,EC-Earth-Consortium,EC-Earth3,ssp585,r9i1p1f1,day,tas,gr,gs://cmip6/CMIP6/ScenarioMIP/EC-Earth-Consorti...,NaN,20200201
2,CMIP,EC-Earth-Consortium,EC-Earth3,historical,r9i1p1f1,day,tas,gr,gs://cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-E...,NaN,20200201
3,ScenarioMIP,EC-Earth-Consortium,EC-Earth3,ssp585,r11i1p1f1,day,tas,gr,gs://cmip6/CMIP6/ScenarioMIP/EC-Earth-Consorti...,NaN,20200201
4,CMIP,EC-Earth-Consortium,EC-Earth3,historical,r6i1p1f1,day,tas,gr,gs://cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-E...,NaN,20200201
...,...,...,...,...,...,...,...,...,...,...,...
124,CMIP,EC-Earth-Consortium,EC-Earth3,historical,r17i1p1f1,day,tas,gr,gs://cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-E...,NaN,20210120
125,CMIP,EC-Earth-Consortium,EC-Earth3,historical,r18i1p1f1,day,tas,gr,gs://cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-E...,NaN,20210121
126,CMIP,EC-Earth-Consortium,EC-Earth3,historical,r21i1p1f1,day,tas,gr,gs://cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-E...,NaN,20210223
127,CMIP,EC-Earth-Consortium,EC-Earth3,historical,r19i1p1f1,day,tas,gr,gs://cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-E...,NaN,20210305


In [22]:
# load data into xarray datasets
dset = cat.to_dataset_dict(zarr_kwargs={'consolidated':True}, storage_options={"anon": True});


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


In [23]:
keys = sorted(dset.keys())

In [24]:
hist = dset[keys[0]]
future = dset[keys[1]]

In [25]:
lat=41.3
lon=-72.5
lon = lon % 360

In [26]:
hist = hist.sel(lat=lat,lon=lon,method='nearest')
future = future.sel(lat=lat,lon=lon,method='nearest')

In [27]:
hist = hist.chunk({'member_id': 1, 'time': -1})
future = future.chunk({'member_id': 1, 'time': -1})

In [28]:
future

<xarray.Dataset>
Dimensions:    (bnds: 2, member_id: 58, time: 31411)
Coordinates:
    lat        float64 41.05
    height     float64 ...
    lat_bnds   (bnds) float64 dask.array<chunksize=(2,), meta=np.ndarray>
    lon        float64 287.6
    lon_bnds   (bnds) float64 dask.array<chunksize=(2,), meta=np.ndarray>
  * time       (time) datetime64[ns] 2015-01-01T12:00:00 ... 2100-12-31T12:00:00
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(31411, 2), meta=np.ndarray>
  * member_id  (member_id) <U10 'r6i1p1f1' 'r9i1p1f1' ... 'r4i1p1f1' 'r3i1p1f1'
Dimensions without coordinates: bnds
Data variables:
    tas        (member_id, time) float32 dask.array<chunksize=(1, 31411), meta=np.ndarray>
Attributes: (12/49)
    source:                             EC-Earth3 (2019): \naerosol: none\nat...
    variable_id:                        tas
    parent_activity_id:                 CMIP
    parent_experiment_id:               historical
    contact:                            cmip6-data@ec-earth.org
    comment:                            This experiment is part of SMHI's Lar...
    ...                                 ...
    grid:                               T255L91
    mip_era:                            CMIP6
    grid_label:                         gr
    parent_time_units:                  days since 1850-01-01
    branch_method:                      standard
    intake_esm_dataset_key:             ScenarioMIP.EC-Earth-Consortium.EC-Ea...

In [29]:
hist

<xarray.Dataset>
Dimensions:    (time: 60266, bnds: 2, member_id: 71)
Coordinates:
    lat        float64 41.05
  * time       (time) datetime64[ns] 1849-12-31T12:00:00 ... 2014-12-31T12:00:00
    height     float64 ...
    lat_bnds   (bnds) float64 dask.array<chunksize=(2,), meta=np.ndarray>
    lon        float64 287.6
    lon_bnds   (bnds) float64 dask.array<chunksize=(2,), meta=np.ndarray>
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(60266, 1), meta=np.ndarray>
  * member_id  (member_id) <U10 'r9i1p1f1' 'r6i1p1f1' ... 'r19i1p1f1'
Dimensions without coordinates: bnds
Data variables:
    tas        (member_id, time) float32 dask.array<chunksize=(1, 60266), meta=np.ndarray>
Attributes: (12/53)
    source:                    EC-Earth3 (2019): \naerosol: none\natmos: IFS ...
    variable_id:               tas
    parent_activity_id:        CMIP
    parent_experiment_id:      piControl
    contact:                   cmip6-data@ec-earth.org
    comment:                   This experiment is part of SMHI's Large Ensemb...
    ...                        ...
    grid:                      ORCA1T255
    mip_era:                   CMIP6
    grid_label:                gr
    parent_time_units:         days since 1850-01-01
    branch_method:             standard
    intake_esm_dataset_key:    CMIP.EC-Earth-Consortium.EC-Earth3.historical....

In [62]:
%%time
ds_future = []
for member_id in future.member_id:
    ds = future.sel(member_id = member_id).load()
    ds_future.append(ds)
future = xr.concat(ds_future,'member_id')

CPU times: user 53min 50s, sys: 18min 16s, total: 1h 12min 7s
Wall time: 33min 9s


In [14]:
np.isfinite(future.tas).sum('time')

<xarray.DataArray 'tas' (member_id: 58)>
dask.array<sum-aggregate, shape=(58,), dtype=int64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
    lat        float64 41.05
    height     float64 ...
    lon        float64 287.6
  * member_id  (member_id) <U10 'r6i1p1f1' 'r9i1p1f1' ... 'r4i1p1f1' 'r3i1p1f1'

In [63]:
future

<xarray.Dataset>
Dimensions:    (bnds: 2, member_id: 58, time: 31411)
Coordinates:
    lat        float64 41.05
    height     float64 2.0
    lat_bnds   (bnds) float64 40.7 41.4
    lon        float64 287.6
    lon_bnds   (bnds) float64 287.2 287.9
  * time       (time) datetime64[ns] 2015-01-01T12:00:00 ... 2100-12-31T12:00:00
    time_bnds  (time, bnds) datetime64[ns] 2015-01-01 2015-01-02 ... 2101-01-01
  * member_id  (member_id) <U10 'r6i1p1f1' 'r9i1p1f1' ... 'r4i1p1f1' 'r3i1p1f1'
Dimensions without coordinates: bnds
Data variables:
    tas        (member_id, time) float32 278.9 284.6 279.1 ... 290.7 282.2 286.8
Attributes: (12/49)
    comment:                            This experiment is part of SMHI's Lar...
    mip_era:                            CMIP6
    status:                             2021-07-04;created; by gcs.cmip6.ldeo...
    latest_applied_cmor_fixer_version:  v3.0
    physics_index:                      1
    grid_label:                         gr
    ...                                 ...
    forcing_index:                      1
    parent_source_id:                   EC-Earth3
    source:                             EC-Earth3 (2019): \naerosol: none\nat...
    product:                            model-output
    table_id:                           day
    intake_esm_dataset_key:             ScenarioMIP.EC-Earth-Consortium.EC-Ea...

In [30]:
%%time
ds_hist = []
for member_id in hist.member_id:
    ds = hist.sel(member_id = member_id).load()
    ds_hist.append(ds)
EC_earth_hist = xr.concat(ds_hist,'member_id')

CPU times: user 2h 22min 18s, sys: 58min 14s, total: 3h 20min 33s
Wall time: 1h 41min 33s


In [31]:
EC_earth_hist

<xarray.Dataset>
Dimensions:    (time: 60266, bnds: 2, member_id: 71)
Coordinates:
    lat        float64 41.05
  * time       (time) datetime64[ns] 1849-12-31T12:00:00 ... 2014-12-31T12:00:00
    height     float64 2.0
    lat_bnds   (bnds) float64 nan nan
    lon        float64 287.6
    lon_bnds   (bnds) float64 287.2 287.9
    time_bnds  (time, bnds) datetime64[ns] NaT NaT ... 2014-12-31 2015-01-01
  * member_id  (member_id) <U10 'r9i1p1f1' 'r6i1p1f1' ... 'r19i1p1f1'
Dimensions without coordinates: bnds
Data variables:
    tas        (member_id, time) float32 nan nan nan nan ... 273.4 284.8 278.8
Attributes: (12/53)
    source:                    EC-Earth3 (2019): \naerosol: none\natmos: IFS ...
    variable_id:               tas
    parent_activity_id:        CMIP
    parent_experiment_id:      piControl
    contact:                   cmip6-data@ec-earth.org
    comment:                   This experiment is part of SMHI's Large Ensemb...
    ...                        ...
    grid:                      ORCA1T255
    mip_era:                   CMIP6
    grid_label:                gr
    parent_time_units:         days since 1850-01-01
    branch_method:             standard
    intake_esm_dataset_key:    CMIP.EC-Earth-Consortium.EC-Earth3.historical....

In [32]:
np.isfinite(EC_earth_hist.tas).sum('time')

<xarray.DataArray 'tas' (member_id: 71)>
array([    0,     0,     0,     0,     0, 60265, 60265, 60265,     0,
       60265, 60265, 60265,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
       60265, 60265, 60265, 60265, 60265, 60265, 60265, 60265])
Coordinates:
    lat        float64 41.05
    height     float64 2.0
    lon        float64 287.6
  * member_id  (member_id) <U10 'r9i1p1f1' 'r6i1p1f1' ... 'r19i1p1f1'

In [33]:
EC_earth_sliced = EC_earth_hist.sel(time=slice('1920','2014'))

In [34]:
np.isfinite(EC_earth_sliced.tas).sum('time')

<xarray.DataArray 'tas' (member_id: 71)>
array([    0,     0,     0,     0,     0, 34699, 34699, 34699,     0,
       34699, 34699, 34699,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
       34699, 34699, 34699, 34699, 34699, 34699, 34699, 34699])
Coordinates:
    lat        float64 41.05
    height     float64 2.0
    lon        float64 287.6
  * member_id  (member_id) <U10 'r9i1p1f1' 'r6i1p1f1' ... 'r19i1p1f1'

In [ ]:
if hist.lon > 180 or hist.lon < -180:
    # convert lon from 0-360 to -180 to 180
    hist = hist.assign_coords(lon=((hist.lon + 180) % 360 - 180))
else: 
    pass 

if future.lon > 180 or future.lon < -180:
    # convert lon from 0-360 to -180 to 180
    future = future.assign_coords(lon=((future.lon + 180) % 360 - 180))
else: 
    pass 

In [ ]:
hist = drop_bounds_height(hist)
future = drop_bounds_height(future)